# PyODB Performance Tests


In [1]:
import cProfile
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from time import sleep

from src.pyodb.pyodb import PyODB, PyODBCache
from test.test_models.complex_models import ComplexBasic, ComplexContainer, ComplexMulti
from test.test_models.primitive_models import PrimitiveBasic, PrimitiveContainer

In [ ]:
def test_insert_performance():
    pyodb = PyODB(2)
    pyodb.add_type(ComplexBasic)
    pyodb.save(ComplexContainer())
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10)):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])

        pyodb.delete(ComplexBasic).gt(random_number = 0).commit()
        pyodb.remove_type(ComplexContainer)
        pyodb.add_type(ComplexContainer)
    del pyodb


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_insert_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
cProfile.run("test_insert_performance()", filepath.as_posix())

In [ ]:
def insert_base_data():
    pyodb = PyODB(persistent=True)
    pyodb.add_type(ComplexBasic)
    pyodb.add_type(ComplexContainer)
    pyodb.add_type(ComplexMulti)

    for _ in tqdm(range(10), desc="Inserting base data"):
        pyodb.save_multiple([ComplexBasic() for _ in range(100)])
        pyodb.save_multiple([ComplexContainer() for _ in range(10)])
        pyodb.save_multiple([ComplexMulti() for _ in range(100)])


def test_select_performance():
    pyodb = PyODB()
    for _ in tqdm(range(100), desc="Testing PrimitiveBasic"):
        pyodb.select(PrimitiveBasic).all()
    for _ in tqdm(range(100), desc="Testing PrimitiveContainer"):
        pyodb.select(PrimitiveContainer).all()
    for _ in tqdm(range(100), desc="Testing ComplexBasic"):
        pyodb.select(ComplexBasic).all()
    for _ in tqdm(range(100), desc="Testing ComplexContainer"):
        pyodb.select(ComplexContainer).all()


base_path = Path(".profile/")
base_path.mkdir(755, exist_ok=True)

filepath = base_path / f"profile_select_{datetime.now().strftime('%y.%m.%d-%H.%M.%S')}.prof"
insert_base_data()
cProfile.run("test_select_performance()", filepath.as_posix())

In [2]:
cache = PyODBCache(PyODB(1))
cache.add_cache("test", lambda x: [PrimitiveBasic() for _ in range(x)], PrimitiveBasic, 2)
print(cache.get_data("test", 3))
print(cache["test"])
sleep(2)
print(cache.get_data("test", 2))
del cache

[PrimitiveBasic: -789, 884.8218501676095, 'i:Zgi=!G,|=9=cRhK^(g>9T f+</z$#'hpkv§FXHB0_z]qxleZZ0RN(§au$<a~v-0n)fMw=^+Xv}`r9:´Uo4wtV', False, PrimitiveBasic: 620, None, ')S8n`H', False, PrimitiveBasic: 499, None, 'kwEE.f0°2^Oa SBA/dZjO,*vwZ|CW4eWnR{FHT`>ZD4pCtd9Q[4n$:B.csT3DF t&E#K&keYGV[5`|fMf-pBYO$z`})XH3v[', False]
[PrimitiveBasic: -789, 884.8218501676095, 'i:Zgi=!G,|=9=cRhK^(g>9T f+</z$#'hpkv§FXHB0_z]qxleZZ0RN(§au$<a~v-0n)fMw=^+Xv}`r9:´Uo4wtV', False, PrimitiveBasic: 620, None, ')S8n`H', False, PrimitiveBasic: 499, None, 'kwEE.f0°2^Oa SBA/dZjO,*vwZ|CW4eWnR{FHT`>ZD4pCtd9Q[4n$:B.csT3DF t&E#K&keYGV[5`|fMf-pBYO$z`})XH3v[', False]
[PrimitiveBasic: -510, 725.3224373959117, 'g4/W[h:IjoRB6k=KvD2QieRS´=', False, PrimitiveBasic: 239, 28.00201143616521, 'r#~_]g*x-°3DV9*_R`!`>}ct'/+^xu*M/(JeiJ^="BP^Im~*Fpt&\~doskt<IHyj3D[2Z"W((t6/´h;j°', True]
